# 感情分析

In [2]:
import torch

print(torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

2.0.1+cu117
cuda:0


## Fine-tuning

In [4]:
from transformers import BertForSequenceClassification, BertTokenizerFast, BertJapaneseTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import pandas as pd

base_model_name = 'cl-tohoku/bert-base-japanese-v3'
# base_model_name = 'cl-tohoku/bert-large-japanese-v2'
prefix = base_model_name.split('/')[-1]

id2tag = pd.read_csv('data/カテゴリー・タグ定義.csv', encoding='cp932')['タグ'].to_dict()
tag2id = {v:k for k, v in id2tag.items()}

model = BertForSequenceClassification.from_pretrained(
    base_model_name, num_labels=len(id2tag), id2label=id2tag, label2id=tag2id)
model = model.to(device)
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-large-japanese-v2')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [6]:
from datasets import load_dataset

dataset = load_dataset('dataset_loader.py', name='category-tagging')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [7]:
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

train_dataset, test_dataset = dataset['train'].map(tokenize, batched=True), dataset['test'].map(tokenize, batched=True)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/536 [00:00<?, ? examples/s]

Map:   0%|          | 0/268 [00:00<?, ? examples/s]

In [8]:
train_dataset, test_dataset

(Dataset({
     features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 536
 }),
 Dataset({
     features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 268
 }))

In [9]:
# Poetryが入っていないとログが出力されないので注意
# !pip install tensorboard/ poetry add tensorboard

# トレーニングの設定
training_args = TrainingArguments(
    output_dir='./results',             # 出力フォルダ
    logging_dir='./logs',               # ログ保存フォルダ
    num_train_epochs=15,               # エポック数
    per_device_train_batch_size=1,      # 訓練のバッチサイズ (GPU数によって変える) 8, 1
    per_device_eval_batch_size=4,      # 評価のバッチサイズ (GPU数によって変える) 16 ,4
    gradient_accumulation_steps=2,      # accumulate gradients over 2 batches (GPU数によって変える)
    warmup_steps=500,                   # 学習率スケジューラのウォームアップステップ数
    weight_decay=0.01,                  # 重み減衰の強さ
    save_steps=1000,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_steps=100,
    prediction_loss_only=True,
)

training_args.output_dir = f'./results_{prefix}_{training_args.num_train_epochs}_v1' # 出力フォルダ

# トレーナーの初期化とトレーニング開始
trainer = Trainer(
    model=model,                        # モデル
    args=training_args,                 # 訓練引数
    train_dataset=train_dataset,        # 訓練データセット
    eval_dataset=test_dataset,          # 評価データセット
    compute_metrics=compute_metrics
)

# チェックポイントから学習を再開したいとき
# trainer.train(ignore_keys_for_eval=['last_hidden_state', 'hidden_states', 'attentions'],
            #   resume_from_checkpoint=True)

trainer.train()

  0%|          | 0/4020 [00:00<?, ?it/s]

{'loss': 3.7732, 'learning_rate': 1e-05, 'epoch': 0.37}


  0%|          | 0/67 [00:00<?, ?it/s]

{'eval_loss': 3.2505757808685303, 'eval_runtime': 2.5704, 'eval_samples_per_second': 104.263, 'eval_steps_per_second': 26.066, 'epoch': 0.37}
{'loss': 2.8241, 'learning_rate': 2e-05, 'epoch': 0.75}


  0%|          | 0/67 [00:00<?, ?it/s]

...

  0%|          | 0/67 [00:00<?, ?it/s]

{'eval_loss': 2.421081066131592, 'eval_runtime': 2.645, 'eval_samples_per_second': 101.324, 'eval_steps_per_second': 25.331, 'epoch': 14.55}
{'loss': 0.0081, 'learning_rate': 2.840909090909091e-07, 'epoch': 14.93}


  0%|          | 0/67 [00:00<?, ?it/s]

{'eval_loss': 2.421330213546753, 'eval_runtime': 2.6489, 'eval_samples_per_second': 101.174, 'eval_steps_per_second': 25.294, 'epoch': 14.93}
{'train_runtime': 520.1544, 'train_samples_per_second': 15.457, 'train_steps_per_second': 7.728, 'train_loss': 0.5909803340999197, 'epoch': 15.0}


TrainOutput(global_step=4020, training_loss=0.5909803340999197, metrics={'train_runtime': 520.1544, 'train_samples_per_second': 15.457, 'train_steps_per_second': 7.728, 'train_loss': 0.5909803340999197, 'epoch': 15.0})

In [10]:
num_training_steps = train_dataset.num_rows / (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs

num_training_steps

4020.0

In [11]:
trainer.evaluate(test_dataset)

  0%|          | 0/67 [00:00<?, ?it/s]

{'eval_loss': 2.421374559402466,
 'eval_runtime': 2.9094,
 'eval_samples_per_second': 92.115,
 'eval_steps_per_second': 23.029,
 'epoch': 15.0}

In [12]:
trainer.save_state()
trainer.save_model()

In [13]:
training_args.output_dir

'./results_bert-base-japanese-v3_15_v1'

In [ ]:
# poetry shell
# tensorboard --logdir ./datasets/sentiment_classification/logs

## 保存したモデルで予測する

In [14]:
from transformers import BertForSequenceClassification, BertTokenizerFast, BertJapaneseTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-v3')
dataset = load_dataset('dataset_loader.py', name='category-tagging')

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

eval_dataset =  dataset['validation'].map(tokenize, batched=True)
eval_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/269 [00:00<?, ? examples/s]

In [16]:
# 保存したモデルを読み込む
model_path = training_args.output_dir
model = BertForSequenceClassification.from_pretrained(model_path)

trainer = Trainer(
    model=model
)

In [17]:
predictions = trainer.predict(eval_dataset)
predictions

  0%|          | 0/34 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[-0.9863571 , -0.13852763, -1.7837257 , ..., -0.19132227,
        -0.7434132 , -0.92474407],
       [-0.69445306, -0.62088645, -1.3496959 , ..., -0.14248541,
        -1.6268102 , -0.26580027],
       [-0.3526515 , -0.22929403, -1.0686407 , ..., -1.1821756 ,
        -0.63832587, -0.5525821 ],
       ...,
       [-2.17435   ,  0.01401598, -1.4897481 , ..., -1.4572196 ,
        -0.58419603, -1.2601547 ],
       [-1.646641  , -0.2593412 , -1.3778213 , ..., -0.8424611 ,
        -0.99090457, -0.69828653],
       [-0.36242336, -0.7550067 ,  0.42984337, ...,  0.7644263 ,
         0.77423805,  0.47332215]], dtype=float32), label_ids=array([40, 41, 40, 30, 30, 41, 40, 40,  8, 45, 40, 39, 39, 41, 40, 40, 52,
       45, 30, 52, 17, 40, 40, 30, 43, 41, 43, 30, 40, 36, 36, 41, 40, 40,
       40, 41, 48, 45, 41, 43, 42, 52, 40,  3, 41, 41, 41, 15, 41, 40, 40,
       40, 46, 40, 30, 43, 40, 40, 44, 52, 40, 45, 30, 41, 45, 40, 33, 30,
       45, 45, 40, 44, 29, 30, 4

In [11]:
eval_dataset.to_pandas()

,text,label,input_ids,token_type_ids,attention_mask
0,以前より高くなっている 。,0,"[2, 13204, 12505, 14031, 12493, 456, 12483, 38...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, ..."
1,小岩井の生乳100％ヨーグルトを安くしてくださって嬉しいです これからも買い続けるのでお値段...,1,"[2, 1829, 7718, 7641, 464, 3904, 7507, 12915, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,ＱＵＩＣpayで支払いで10%OＦＦにするならば、majicaにチャージして支払いをした場合...,1,"[2, 64, 7100, 13952, 7085, 13243, 457, 19567, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,9月の特売品がたくさん有って良かったです!,1,"[2, 40, 2806, 464, 3720, 7932, 1286, 430, 2227...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,是非気軽にいただける飲食店を増やしてほしいです！フードコートでも店舗でも。,1,"[2, 32039, 3235, 7316, 461, 27184, 12685, 1901...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...
320,カツカレーライスを、購入したのだけど、フォークと割り箸しか無くて、残念ですスプーンも有ると嬉...,0,"[2, 21804, 19487, 25281, 500, 384, 13929, 441,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
321,ナシ。,2,"[2, 546, 7033, 385, 3, 0, 0, 0, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
322,手首を痛めているので、店員さんにカゴの移動をお願いしたら、笑顔で運んでくれ､とても感じ良かっ...,1,"[2, 29091, 500, 24031, 456, 12483, 464, 457, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
323,会計時にSSさんがLINEのお友達登録を勧めて下さりスマホの登録の方法も丁寧に教えてくれまし...,1,"[2, 17643, 2734, 461, 16104, 13038, 430, 25426...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
